In [ ]:
!pip install gensim nltk

In [3]:

with open('Action Bronson_lyrics.txt', 'r', encoding='utf-8') as file:
    bronson_text = file.read() #change w new rapper

print(bronson_text[:500])



Heartbreak drowned sorrows in a large steak

Why you always all on my back?
Why you gotta do me like that?
Why you gotta act like a bitch when I'm with you?
Baby girl I'm blue

Because you treat me like shit
I paid for the bed and never even slept in it
I paid for that crib I never stepped foot in
And now somebody else is eating all the pudding
Things change, now my dashboard wooden
All black Benz, like a young Doc Gooden
Dark shades, cause I'm stone crazy
Girl, we grown, stop playing on my pho


In [5]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download('punkt')

# sentences
sentences = sent_tokenize(bronson_text)

# words
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]


print(tokenized_sentences[0])





['heartbreak', 'drowned', 'sorrows', 'in', 'a', 'large', 'steak', 'why', 'you', 'always', 'all', 'on', 'my', 'back', '?']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
from google.colab import files
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

# tokenizing files
def tokenize_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        tokens = word_tokenize(text)
    return tokens

# Upload files
uploaded = files.upload()

all_tokens = []

#iterating
for filename in uploaded.keys():
    tokens = tokenize_file(filename)
    all_tokens.append(tokens)

# training stuff
model = Word2Vec(sentences=all_tokens, vector_size=100, window=5, min_count=1, workers=4)

model.save("rap_lyrics_word2vec.model")

# Formatting
output_file = "glove_appropriate_vectors.txt"
with open(output_file, 'w', encoding='utf-8') as f:
    # header stuff
    vocab_size = len(model.wv)
    vector_size = model.vector_size
    f.write(f"{vocab_size} {vector_size}\n")

    for word in model.wv.index_to_key:
        vector = model.wv[word]
        vector_str = ' '.join(map(str, vector))
        f.write(f"{word} {vector_str}\n")


files.download(output_file)


Saving Action Bronson_lyrics.txt to Action Bronson_lyrics (1).txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
!pip install gensim pyphen



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.4 MB/s eta 0:00:00


In [9]:
import numpy as np
from gensim.models import KeyedVectors
import requests
import random
import string
import nltk
from nltk.corpus import wordnet as wn

# natty lang toolkit
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [18]:

file_path = 'glove_appropriate_vectors.txt'

# being cleaned
def clean_word(word):
    cleaned = word.translate(str.maketrans('', '', string.punctuation))
    return cleaned if len(cleaned) > 2  else ''

#loading
def load_vectors(file_path):
    with open(file_path, 'r') as f:
        num_vectors, vector_size = map(int, f.readline().split())
        word_vectors = KeyedVectors(vector_size)

        for line in f:
            values = line.split()
            word = clean_word(values[0])
            if word:  # not null
                vector = np.asarray(values[1:], dtype='float32')
                word_vectors.add_vector(word, vector)

    return word_vectors

word_vectors = load_vectors(file_path)

# Datamuse stuff
def get_rhyming_words(word, topics="justice,drugs,alcohol,love", num_words=5000):
    url = f"https://api.datamuse.com/words?rel_rhy={word}&max={num_words}&topics={topics}"
    response = requests.get(url)
    rhymes = response.json()
    rhyming_words = [clean_word(rhyme['word']) for rhyme in rhymes if clean_word(rhyme['word']) in word_vectors.key_to_index]
    return [word for word in rhyming_words if len(word) > 2]  # semantic trouble w short words

# POS tagging
def get_word_with_pos(word, pos_tag, used_words, context=[]):
    similar_words = word_vectors.most_similar(word, topn=1000)
    for similar_word, _ in similar_words:
        cleaned_word = clean_word(similar_word)
        if cleaned_word and nltk.pos_tag([cleaned_word])[0][1] == pos_tag and cleaned_word not in used_words and cleaned_word not in context:
            return cleaned_word
    for similar_word, _ in similar_words:
        cleaned_word = clean_word(similar_word)
        if cleaned_word and cleaned_word not in used_words:
            return cleaned_word
    return word

# basic sentence templates + context
def generate_line(template, used_words, context=[]):
    tokens = template.split()
    line = []
    for token in tokens:
        pos_tag = token
        if pos_tag == 'NN':
            word = get_word_with_pos(random.choice(context) if context else 'word', 'NN', used_words, context)
        elif pos_tag == 'VB':
            word = get_word_with_pos(random.choice(context) if context else 'word', 'VB', used_words, context)
        else:
            word = token
        line.append(word)
        used_words.add(word)
        context.append(word)
    return " ".join(line)

# generating templates
def generate_verse(start_word, topics=None, num_lines=20, max_line_length=10, rhyme_scheme='AABB'):
    verse = []
    current_word = clean_word(start_word)
    used_words = set()
    templates = [
        "NN VB NN",
        "NN VB NN with NN",
        "NN VB NN and NN VB NN",
        "NN VB NN because NN VB NN"

    ]

    for _ in range(num_lines // 2):
        template1 = random.choice(templates)
        line1 = generate_line(template1, used_words, [current_word])

        rhyming_words1 = get_rhyming_words(current_word, topics)
        if rhyming_words1:
            rhyming_word1 = random.choice(rhyming_words1)
            line1 += f" with {rhyming_word1}"
            used_words.add(rhyming_word1)
            current_word = clean_word(rhyming_word1)
        else:
            rhyming_word1 = current_word

        verse.append(line1)

        template2 = random.choice(templates)
        line2 = generate_line(template2, used_words, [current_word])

        rhyming_words2 = get_rhyming_words(current_word, topics)
        if rhyming_words2:
            rhyming_word2 = random.choice(rhyming_words2)
            line2 += f" with {rhyming_word2}"
            used_words.add(rhyming_word2)
            current_word = clean_word(rhyming_word2)
        else:
            rhyming_word2 = current_word

        verse.append(line2)

    return "\n".join(verse)

start_word = "food"
topics = ["food", "cars"]
verse = generate_verse(start_word,topics=topics) #incl start_word here if you want to uncomment
print(verse)

with open("generated_verse_kl.txt", "w") as f:
    f.write(verse)


ace Tap Scion with Lenny with longitude
drunk keep hair and bullshit get shit with food
Came Welcome Fila and fence Slow kingdom with dude
play make man because bitch have fuck with nude
sticky remember glass with rude
Notre Belize Medication because Hello take Uhh with nude
vehicle bullpen attack with longitude
pass see land because Jungle give time with food
gavel Kind die with longitude
dropkick know dick with attitude
beach GIVE hung and Bronco Throw pirouette with rude
frame remove east with nude
Piston Keep kitty and llac say Russia with food
height Pippen Medikit and Fuck let flipjack with rude
yakitori Reverend Nevada and Stick leave brag with nude
Pack Look Justin with dude
Cause Take baby with longitude
day believe Ride because child Put head with attitude
truant Shake hoody with dude
smoke Get money and Shit slow want with food
